In [1]:
# read the data

text = []

with open('mingshilu_texts.txt','r') as f:
    for line in f.read().splitlines():
        if line != '':
            text.append(line)
print 'There are',len(text),'text entries'

There are 71714 text entries


In [2]:
# for now, experiment with the first 10,000 entries

text = text[:10000]

# convert to unicode
for i in range(len(text)):
    text[i] = text[i].decode('utf8')

In [3]:
# extract the unique tokens

tokens_dict = {}
reverse_tokens_dict = {}

counter = 0
for document in text:
    for token in document:
        if token not in tokens_dict:
            tokens_dict[token] = counter
            reverse_tokens_dict[counter] = token
            counter += 1

print counter

5406


In [4]:
# tokenize each document

tokenized_text = []
for document in text:
    tokenized_document = []
    for token in document:
        tokenized_document.append(tokens_dict[token])
    tokenized_text.append(tokenized_document)

In [5]:
# set the priors
import numpy as np

topic_num = 50
token_num = len(tokens_dict)
doc_num = len(tokenized_text)

total_tokens = 0
for document in tokenized_text:
    total_tokens += len(document)

alpha, beta, gamma, delta = 1, 0.01, 0.1, 0.01

In [6]:
# initialize the topics and the counts

# def initialize():
topics = {}
bigram_status = {}
# how many times word w is assigned into topic z as a unigram
n_zw = np.zeros((topic_num, token_num))
# how many times word v is assigned to topic z as the 2nd term of a bigram given the previous word w
m_zwv = np.zeros((topic_num, token_num, token_num))
# how many times the status variable x = k (0 or 1) given the previous word w and the previous word's topic z
p_zwk = np.zeros((topic_num, token_num, 2))
# how many times a word is assigned to z in document d
q_dz = np.zeros((doc_num, topic_num))
# how many times a word is assigned to z as a unigram
n_z = np.zeros(topic_num)
# how many times a word is assigned to z and previous word w as a bigram
m_zw = np.zeros((topic_num, token_num))
    
#     for m, document in enumerate(tokenized_text):
#         previous_z = -1
#         for i, token in enumerate(document):
#             z = np.random.randint(topic_num)
#             if i==0:
#                 x = 0
#                 n_zw[z,token] += 1
#             else:
#                 x = np.random.randint(2)
#                 if x==0:
#                     n_zw[z,token] += 1
#                 if x==1:
#                     last_token = document[i-1]
#                     m_zwv[z,last_token,token] += 1
#                 p_zmk[previous_z,last_token,x] += 1
#             q_dz[m,z] += 1
#             topics[(m,i)] = z
#             bigram_status[(m,i)] = x
    
#     return topics, bigram_status, n_zw, m_zwv, p_zmk, q_dz

In [7]:
for d, document in enumerate(tokenized_text):
    previous_z = -1
    for i, token in enumerate(document):
        z = np.random.randint(topic_num)
        if i == 0:
            x = 0
            n_zw[z,token] += 1
            n_z[z] += 1
        else:
            x = np.random.randint(2)
            last_token = document[i-1]
            if x==0:
                n_zw[z,token] += 1
                n_z[z] += 1
            if x==1:
                m_zwv[z,last_token,token] += 1
                m_zw[z,last_token] += 1
            p_zwk[previous_z,last_token,x] += 1
        previous_z = z
        q_dz[d,z] += 1
        topics[(d,i)] = z
        bigram_status[(d,i)] = x

In [8]:
def conditional_distribution_for_z(x, token, d, last_token):
    first_term = alpha + q_dz[d,:]
    if x == 0:
        numerator = beta + n_zw[:,token]
        denominator = token_num*beta + n_z
    if x == 1:
        numerator = delta + m_zwv[:,last_token,token]
        denominator = token_num*delta + m_zw[:,last_token]
    p_of_z = numerator/denominator*first_term
    p_of_z /= np.sum(p_of_z)
    return p_of_z

In [9]:
def conditional_distribution_for_x(z, token, last_token, previous_z):
    if last_token == -1:
        return [1,0]
    first_term_0 = gamma + p_zwk[previous_z, last_token, 0]
    first_term_1 = gamma + p_zwk[previous_z, last_token, 1]
    numerator_0 = beta + n_zw[z,token]
    denominator_0 = token_num*beta + n_z[z]
    numerator_1 = delta + m_zwv[z,last_token,token]
    denominator_1 = token_num*delta + m_zw[z,last_token]
    p_of_z_0 = numerator_0/denominator_0*first_term_0
    p_of_z_1 = numerator_1/denominator_1*first_term_1
    p_of_z = [p_of_z_0,p_of_z_1]/(p_of_z_0+p_of_z_1)
    return p_of_z

In [10]:
from scipy.special import gammaln

def loglikelihood():
    lik = 0
    prior_beta = gammaln(np.sum(np.ones(token_num)*beta))-np.sum(gammaln(np.ones(token_num)*beta))
    prior_delta = gammaln(np.sum(np.ones(token_num)*delta))-np.sum(gammaln(np.ones(token_num)*delta))
    prior_gamma = gammaln(np.sum(np.ones(2)*gamma))-np.sum(gammaln(np.ones(2)*gamma))
    prior_alpha = gammaln(np.sum(np.ones(topic_num)*alpha))-np.sum(gammaln(np.ones(topic_num)*alpha))
    for z in range(topic_num):
        lik += (np.sum(gammaln(n_zw[z,:]+beta)) - gammaln(np.sum(n_zw[z,:]+beta)) + prior_beta)/total_tokens
        for w in range(token_num):
            lik += (np.sum(gammaln(m_zwv[z,w,:]+delta)) - gammaln(np.sum(m_zwv[z,w,:]+delta)) + prior_delta)/total_tokens
            lik += (np.sum(gammaln(p_zwk[z,w,:]+gamma)) - gammaln(np.sum(p_zwk[z,w,:]+gamma)) + prior_gamma)/total_tokens
    for d in range(doc_num):
        lik += (np.sum(gammaln(q_dz[d,:]+alpha)) - gammaln(np.sum(q_dz[d,:]+alpha)) + prior_alpha)/total_tokens
    return lik

In [11]:
def gibbs_sampler():
    previous_topics = topics.copy()
    for d,document in enumerate(tokenized_text):
        for i, token in enumerate(document):
            z = topics[(d,i)]
            x = bigram_status[(d,i)]
            last_token = document[i-1]
            if i == 0:
                previous_z = -1
            else:
                previous_z = previous_topics[(d,i-1)]
            if x == 0:
                if n_zw[z,token] < 1:
                    print 'n_zw'
                    print d
                    print z
                    print i
                    return
                n_zw[z,token] -= 1
                n_z[z] -= 1
            if x == 1:
                if m_zwv[z,last_token,token] < 1:
                    print 'm_zwk'
                    print d
                    print z
                    print i
                    return
                m_zwv[z,last_token,token] -= 1
                m_zw[z,last_token] -= 1
            if i != 0:
                if p_zwk[previous_z,last_token,x] < 1:
                    print 'p_zwk'
                    print d
                    print z
                    print i
                    print x
                    print p_zwk[previous_z,last_token,x]
                    print last_token
                    return
                p_zwk[previous_z,last_token,x] -= 1
            if q_dz[d,z] < 1:
                print 'q_dz'
                print d
                print z
                print i
                print x
                return
            q_dz[d,z] -= 1
            
            p_z = conditional_distribution_for_z(x, token, d, last_token)
            p_x = conditional_distribution_for_x(z, token, last_token, previous_z)
            x = np.random.multinomial(1,p_x).argmax()
            z = np.random.multinomial(1,p_z).argmax()
            
            if i == 0:
                previous_z = -1
            else:
                previous_z = topics[(d,i-1)]
            
            if x == 0:
                n_zw[z,token] += 1
                n_z[z] += 1
            if x == 1:
                m_zwv[z,last_token,token] += 1
                m_zw[z,last_token] += 1
            if i != 0:
                p_zwk[previous_z,last_token,x] += 1
            q_dz[d,z] += 1
            
            topics[(d,i)] = z
            bigram_status[(d,i)] = x
        
        if d%1000 == 0:
            print 'done with',d
        
    print loglikelihood()

In [12]:
loglikelihood
for i in range(50):
    gibbs_sampler()

done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-10.8513182612
done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-10.5001722994
done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-10.2732693017
done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-10.1076684734
done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-9.96547839232
done with 0
done with 1000
done with 2000
done with 3000
done with 4000
done with 5000
done with 6000
done with 7000
done with 8000
done with 9000
-9.83389202454
done with 0
done with 1000
d

In [13]:
def get_theta():
    theta = []
    for d in range(doc_num):
        numerator = alpha + q_dz[d,:]
        denominator = doc_num*alpha + np.sum(q_dz[d,:])
        curr_theta = numerator/denominator
        theta.append(curr_theta)
    return theta

In [14]:
def get_phi():
    phi = []
    for z in range(topic_num):
        numerator = beta + n_zw[z,:]
        denominator = topic_num*beta + np.sum(n_zw[z,:])
        curr_phi = numerator/denominator
        phi.append(curr_phi)
    return phi

In [15]:
phi = get_phi()

In [16]:
# get most likely words for each topic

def get_phi_max(phi):
    max_words = []
    for topic in phi:
        curr_list = sorted(range(len(topic)), key=lambda k: topic[k])[-20:]
        curr_list = [reverse_tokens_dict[word] for word in curr_list]
        max_words.append(curr_list)
    return max_words

In [17]:
max_words = get_phi_max(phi)

In [18]:
# look at topic assignment

for i in range(topic_num):
    print sum(n_zw[i,:])

13029.0
37256.0
3782.0
3926.0
29090.0
4389.0
12100.0
3880.0
3887.0
3844.0
7680.0
3854.0
4504.0
4968.0
5467.0
3830.0
3863.0
42801.0
13713.0
14750.0
12956.0
32022.0
3929.0
4177.0
4008.0
5491.0
15855.0
5191.0
3982.0
6535.0
3674.0
3796.0
11591.0
4566.0
6299.0
3981.0
4195.0
25676.0
4140.0
3758.0
42182.0
3921.0
6404.0
3964.0
12367.0
29500.0
4089.0
3995.0
8578.0
4125.0


In [19]:
for i,topic in enumerate(max_words):
    print 'topic',i
    for word in topic:
        print word

topic 0
至
大
无
必
遂
兵
军
出
数
曰
将
矣
其
诸
敌
我
而
不
上
之
topic 1
也
此
一
于
而
今
曰
者
无
有
人
天
所
为
以
其
朕
不
尔
之
topic 2
在
知
大
未
言
非
民
得
及
当
于
为
者
出
以
而
其
上
不
之
topic 3
今
可
不
于
其
居
在
令
者
无
上
时
得
复
奏
言
以
从
乞
民
topic 4
其
改
礼
任
广
陈
俱
故
山
县
吏
监
命
事
于
复
以
升
行
为
topic 5
己
法
死
奏
劾
自
受
至
复
宥
乃
遂
姑
罪
之
其
初
下
上
不
topic 6
四
巡
直
田
设
平
之
安
府
广
城
三
民
江
二
水
山
河
州
县
topic 7
时
在
劳
小
亦
从
事
已
者
宜
言
故
命
先
复
不
有
其
以
之
topic 8
从
时
等
行
山
自
应
又
之
所
命
其
今
为
奏
复
请
言
于
上
topic 9
军
往
自
皆
既
不
如
行
人
请
奏
且
车
遂
之
其
为
以
上
所
topic 10
总
并
将
白
二
有
三
都
钞
百
指
军
人
升
一
赏
等
各
官
者
topic 11
已
何
仍
乞
曰
等
自
太
令
为
命
其
之
从
至
所
先
于
上
以
topic 12
及
三
以
二
又
凡
素
在
等
一
朝
自
命
官
人
各
不
皆
而
行
topic 13
至
府
等
新
伪
平
道
多
交
下
海
镇
三
于
大
太
江
清
安
之
topic 14
上
汝
同
事
所
又
及
王
人
为
与
至
济
言
遣
高
不
其
之
有
topic 15
欲
得
与
乞
复
无
可
而
何
之
其
事
今
所
等
不
从
以
上
奏
topic 16
请
仍
出
从
今
有
先
命
未
无
为
令
当
故
事
其
以
奏
不
所
topic 17
令
二
于
各
岁
给
行
从
所
之
乞
以
其
今
有
不
官
民
者
一
topic 18
归
阿
各
马
命
还
亦
至
并
哈
使
来
国
王
及
其
朝
赐
遣
等
topic 19
张
代
弟
李
通
兴
卫
府
都
王
羽
升
山
指
袭
命
金
俱
为
子
topic 20
亦
矣
为
道
无
学
得
其
教
也
然

In [20]:
def get_sigma():
    max_bigrams = []
    for z in range(topic_num):
        max_sigma = np.zeros((token_num,2))
        for w in range(token_num):
            numerator = delta + m_zwv[z,w,:]
            denominator = topic_num*delta + np.sum(m_zwv[z,w,:])
            curr_sigma = numerator/denominator
            max_sigma = [curr if curr[0]>=curr_sigma[i] else (curr_sigma[i],w) for i,curr in enumerate(max_sigma) ]
        second_words = sorted(range(len(max_sigma)), key=lambda k: max_sigma[k][0])[-20:]
        first_words = [max_sigma[i][1] for i in second_words]
        first_words = [reverse_tokens_dict[word] for word in first_words]
        second_words = [reverse_tokens_dict[word] for word in second_words]
        words = []
        for i in range(20):
            words.append(first_words[i]+second_words[i])
        max_bigrams.append(words)
    return max_bigrams

In [21]:
max_bigrams = get_sigma()

In [22]:
for i,topic in enumerate(max_bigrams):
    print 'topic',i
    for word in topic:
        print word

topic 0
耳灰
都指
夹击
顿首
房昭
燕王
皇考
京城
滹沱
款台
狼狈
德州
总兵
渡河
辎重
谍报
宋忠
炳文
陛下
景隆
topic 1
根本
兀良
医药
叩首
访求
怙终
指挥
睦邻
驸马
暹答
悠久
)[
头目
邻境
钦哉
瓦剌
夙夜
艰难
＜⿱
帖木
topic 2
旗张
勿毁
驻白
度潘
擒械
布司
絷维
充弓
绢叚
蒙颁
荒疏
洪仔
上谕
留之
郝郁
金银
郎沈
百十
右参
指挥
topic 3
管送
款附
效尤
密勒
疾乞
礼币
册黔
性惇
满圈
比者
扰民
铅沙
坐诬
营墩
五顷
惟倚
目那
行在
开煎
指挥
topic 4
训导
春坊
序班
杭州
舍人
欧阳
釒＞
苑马
湖广
翰林
＜⿰
仆寺
佥事
指挥
鸿胪
按察
狭西
布政
浙江
御史
topic 5
甘霖
雨歌
轻减
⿰釒
刻箭
慈浡
衡绞
疋花
谙韬
岑瑛
科劾
佥事
戍威
问罪
俱逮
挥李
御史
指挥
察御
都指
topic 6
绍兴
市桥
渰没
忠义
兖州
皇太
霪雨
春夏
浙江
右参
辉府
麓川
保定
彰德
壬午
宣慰
直隶
赈济
巡检
指挥
topic 7
行在
在工
按法
千余
违制
田最
左角
副刘
暇輙
袭破
弓劲
焦谦
向炎
太淑
督冉
答乜
西巩
挥同
都督
指挥
topic 8
中孙
成志
恤孤
楚世
例应
差辛
休番
矜悯
市薪
闻庚
初崇
袭绢
条奏
衣币
在通
祭署
坛祠
太社
祠祭
都督
topic 9
常存
乃恃
猛虎
玄圭
送狭
纻纱
偏岭
馆谙
严衢
嘉瓜
挥同
行在
靖安
孝先
记忆
脱卜
支麦
从其
指挥
慰司
topic 10
丙辰
接应
奇功
尚书
向前
己卯
减半
绮衣
火长
编伍
仪卫
彩币
织金
舍人
校尉
纻丝
佥事
镇抚
表里
指挥
topic 11
商度
贤能
粮数
汪泰
妄援
空酒
代智
河钦
儿干
场灶
宽猛
定冀
桃园
赵玟
御案
抚养
头崖
一员
行在
指挥
topic 12
读讫
坊厢
黑角
阙设
屠宰
嫁娶
三日
纱帽
听选
宿歇
二十
第四
太孙
善门
斩衰
夕哭
乌纱
角带
指挥
素服
topic 13
桧花
号纪
亭侯
纲司
除凶
超类
奋励
归化
更姓
富良
露布
督佥
阴阳
思郎
芹站
总管
慈廉
洮江
指挥
交州
topic 14
普济
阴蓄
驯象
爱弟
瓦甸
指挥
同谋
太原
谤毁
水峪
宁化
赵王

In [23]:
# get number of unigrams
total_unigram = 0
for i in range(topic_num):
    total_unigram += sum(n_zw[i,:])
print total_unigram

505560.0


In [26]:
# get number of bigrams
total_bigram = 0
for i in range(topic_num):
    for w in range(token_num):
        total_bigram += sum(m_zwv[i,w,:])
print total_bigram

889004.0


In [27]:
total_tokens

1394564